In [12]:
#modules needed for using alpaca and formatting our data
import alpaca_trade_api as tradeapi
import requests  
import json
import pandas as pd
import tensorflow as tf
import numpy as np

In [3]:
api = tradeapi.REST(
    key_id='PKFH5ZA4MRGSTSEFNQ3S',
    secret_key='Yy5ygxRCE6bOwRRC03tQ9mYMudA6WQRKkRMbTnFY',
    base_url='https://paper-api.alpaca.markets'
)

In [10]:
def getData(symbol, timeframe, limit):
    barset = api.get_barset(symbol, timeframe, limit)
    return barset


DATA = getData("ADBE", "day", 5)

In [11]:
#change data pulled into same format as data from quandl so that we may use our trained Model on it
def changeFormat(data):
    o = []
    h = []
    l = []
    c = []
    v = []
    for d in data:
        for i in data[d]:
            o.append(i.o)
            h.append(i.h)
            l.append(i.l)
            c.append(i.c)
            v.append(i.v)
    dataDict = {"open":o, "high":h, "low":l, "close":l, "volume":v}
    df = pd.DataFrame(dataDict)
    return df


formatted_data = changeFormat(DATA)
print(formatted_data)

full_data = len(formatted_data)
print(full_data)

     open      high      low    close   volume
0  449.84  450.7400  442.280  442.280  1274960
1  448.26  449.0799  440.700  440.700  2346534
2  449.09  454.0350  446.345  446.345  2063261
3  451.35  464.3700  449.130  449.130  2063250
4  460.87  461.7900  445.090  445.090  3049001
5


In [6]:
#Testing Model 1 on Alpaca data

#########this works, and should be all we need to test the saved Model 1
dataset = formatted_data.values
data_mean = dataset[:full_data].mean(axis = 0)
data_std = dataset[:full_data].std(axis = 0)

dataset = (dataset-data_mean)/data_std
dataset

array([[-1.08358789, -0.34266158, -1.31382093, -1.31382093,  1.45890181],
       [-0.54150328, -0.87497692, -0.75366472, -0.75366472, -0.31781887],
       [-0.39907891, -1.12496403, -0.17653408, -0.17653408, -1.62260216],
       [ 0.19823148,  1.40320755,  0.84193176,  0.84193176,  0.34523096],
       [ 1.8259386 ,  0.93939498,  1.40208797,  1.40208797,  0.13628826]])

In [7]:
#do multivariate data function on this to further organize for Model 1
##had difficulty importing the below functions from Model_test.ipynb file

######all this function shit may not be necessary, im not sure -- all pulled from Model_test file

def multivariate_data(dataset, target, start_index, end_index, history_size,
                     target_size, step, single_step = False):
    data = []
    labels = []
    
    start_index = start_index+history_size
    if end_index is None:
        end_index = len(dataset) - target_size
        
    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])
        
        if single_step:
            labels.append(dataset[indices])
        else:
            labels.append(target[i:i+target_size])
            
    return np.array(data), np.array(labels)
            

In [8]:
past_history = 1
future_target = 1
STEP = 1

In [13]:
x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1],
                                               0, full_data, past_history,
                                               future_target, STEP,
                                               single_step=True)

In [16]:
BATCH_SIZE = 5
BUFFER_SIZE = full_data

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

In [15]:
############this works and imports our saved model into here

new_model_1 = tf.keras.models.load_model('saved_model_1/original_df_model')

In [24]:
####using processed data 'val_data_single' instead of normalized, raw 'dataset'
predictions = new_model_1.predict(val_data_single, steps=1)
predictions

array([[-0.43825278],
       [-0.44943917],
       [-0.4346034 ],
       [-0.13667868]], dtype=float32)

In [22]:
#dataset = (dataset-data_mean)/data_std

def price(predictions):
    prices = []
    for prediction in predictions:
        price = prediction*data_std+data_mean
        prices.append(price)
    return prices

In [23]:
price(predictions)

[array([4.37240451e+02, 4.45937709e+02, 4.34664896e+02, 4.34664896e+02,
        3.96206590e+07]),
 array([4.37163479e+02, 4.45866560e+02, 4.34625355e+02, 4.34625355e+02,
        3.95045048e+07]),
 array([4.37265562e+02, 4.45960920e+02, 4.34677795e+02, 4.34677795e+02,
        3.96585524e+07]),
 array([4.39315535e+02, 4.47855819e+02, 4.35730878e+02, 4.35730878e+02,
        4.27520600e+07])]